# Evolver Loop 3 Analysis

## Critical Issue: All submission candidates have overlaps!

The evaluator correctly identified that our current submissions would be REJECTED by Kaggle.

**Goal**: Create a valid baseline with 0 overlaps, then continue optimization.

In [1]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree

getcontext().prec = 25
scale_factor = Decimal('1e15')

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

In [2]:
def has_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def get_side_length(trees):
    if not trees:
        return Decimal('0')
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    minx = Decimal(bounds[0]) / scale_factor
    miny = Decimal(bounds[1]) / scale_factor
    maxx = Decimal(bounds[2]) / scale_factor
    maxy = Decimal(bounds[3]) / scale_factor
    return max(maxx - minx, maxy - miny)

def calculate_score(side_lengths):
    score = Decimal('0')
    for n, side in side_lengths.items():
        score += side ** 2 / Decimal(str(n))
    return float(score)

def load_submission(csv_path):
    df = pd.read_csv(csv_path)
    df['x'] = df['x'].astype(str).str.lstrip('s')
    df['y'] = df['y'].astype(str).str.lstrip('s')
    df['deg'] = df['deg'].astype(str).str.lstrip('s')
    
    tree_lists = {}
    side_lengths = {}
    
    for n in range(1, 201):
        prefix = f'{n:03d}_'
        group = df[df['id'].str.startswith(prefix)]
        trees = [ChristmasTree(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]
        tree_lists[n] = trees
        side_lengths[n] = get_side_length(trees)
    
    return tree_lists, side_lengths

In [3]:
# Load best_snapshot (score 70.627582, 2 invalid: N=158, 184)
print("Loading best_snapshot.csv...")
best_trees, best_sides = load_submission('/home/code/preoptimized/best_snapshot.csv')
best_score = calculate_score(best_sides)
print(f"best_snapshot score: {best_score:.6f}")

# Load chistyakov_best (score 70.926150, 0 invalid)
print("\nLoading chistyakov_best.csv...")
chistyakov_trees, chistyakov_sides = load_submission('/home/code/preoptimized/chistyakov_best.csv')
chistyakov_score = calculate_score(chistyakov_sides)
print(f"chistyakov_best score: {chistyakov_score:.6f}")

# Verify invalid N values in best_snapshot
print("\nVerifying invalid N values in best_snapshot...")
for n in [158, 184]:
    if has_overlap(best_trees[n]):
        print(f"  N={n}: HAS OVERLAP (side={float(best_sides[n]):.6f})")
        print(f"    chistyakov N={n}: side={float(chistyakov_sides[n]):.6f}")
    else:
        print(f"  N={n}: valid")

Loading best_snapshot.csv...


best_snapshot score: 70.627582

Loading chistyakov_best.csv...


chistyakov_best score: 70.926150

Verifying invalid N values in best_snapshot...
  N=158: HAS OVERLAP (side=7.343104)
    chistyakov N=158: side=7.374218
  N=184: HAS OVERLAP (side=7.902634)
    chistyakov N=184: side=7.932324


In [4]:
# Create valid baseline by replacing invalid N values
print("Creating valid baseline...")

valid_trees = {}
valid_sides = {}

for n in range(1, 201):
    if n in [158, 184]:  # Use chistyakov for invalid N values
        valid_trees[n] = chistyakov_trees[n]
        valid_sides[n] = chistyakov_sides[n]
    else:
        valid_trees[n] = best_trees[n]
        valid_sides[n] = best_sides[n]

valid_score = calculate_score(valid_sides)
print(f"\nValid baseline score: {valid_score:.6f}")
print(f"Score change from best_snapshot: {valid_score - best_score:.6f}")

# Verify no overlaps
print("\nVerifying no overlaps...")
invalid_count = 0
for n in range(1, 201):
    if has_overlap(valid_trees[n]):
        print(f"  N={n}: HAS OVERLAP")
        invalid_count += 1

if invalid_count == 0:
    print("  All N values are valid! No overlaps.")
else:
    print(f"  {invalid_count} N values have overlaps")

Creating valid baseline...

Valid baseline score: 70.633035
Score change from best_snapshot: 0.005453

Verifying no overlaps...


  All N values are valid! No overlaps.


In [5]:
# Save valid baseline
print("Saving valid baseline...")

rows = []
for n in range(1, 201):
    trees = valid_trees[n]
    for t_idx, tree in enumerate(trees):
        rows.append({
            'id': f'{n:03d}_{t_idx}',
            'x': f's{float(tree.center_x):.12f}',
            'y': f's{float(tree.center_y):.12f}',
            'deg': f's{float(tree.angle):.12f}'
        })

submission_df = pd.DataFrame(rows)
submission_df.to_csv('/home/code/preoptimized/valid_baseline.csv', index=False)
submission_df.to_csv('/home/submission/submission.csv', index=False)
print(f"Saved valid baseline")
print(f"\nFinal score: {valid_score:.6f}")
print(f"Target: 68.896973")
print(f"Gap: {valid_score - 68.896973:.6f} ({(valid_score - 68.896973) / 68.896973 * 100:.2f}%)")

Saving valid baseline...
Saved valid baseline

Final score: 70.633035
Target: 68.896973
Gap: 1.736062 (2.52%)


In [6]:
# Search for better valid solutions for N=158 and N=184 from all preoptimized sources
print("Searching for better valid solutions for N=158 and N=184...")

sources = [
    '/home/code/preoptimized/best_snapshot.csv',
    '/home/code/preoptimized/better_ensemble.csv',
    '/home/code/preoptimized/bucket_of_chump.csv',
    '/home/code/preoptimized/chistyakov_best.csv',
    '/home/code/preoptimized/saspav_best.csv',
    '/home/code/preoptimized/submission_70_926.csv',
]

best_for_n = {158: None, 184: None}
best_side_for_n = {158: float('inf'), 184: float('inf')}

for source in sources:
    trees, sides = load_submission(source)
    for n in [158, 184]:
        if not has_overlap(trees[n]):
            side = float(sides[n])
            if side < best_side_for_n[n]:
                best_side_for_n[n] = side
                best_for_n[n] = (source, trees[n], sides[n])
                print(f"  N={n}: Found better valid solution in {source.split('/')[-1]}: side={side:.6f}")

print(f"\nBest valid solutions:")
for n in [158, 184]:
    if best_for_n[n]:
        source, trees, side = best_for_n[n]
        print(f"  N={n}: {source.split('/')[-1]} with side={float(side):.6f}")

Searching for better valid solutions for N=158 and N=184...


  N=158: Found better valid solution in bucket_of_chump.csv: side=7.343318
  N=184: Found better valid solution in bucket_of_chump.csv: side=7.903088


  N=158: Found better valid solution in saspav_best.csv: side=7.343104
  N=184: Found better valid solution in saspav_best.csv: side=7.902634



Best valid solutions:
  N=158: saspav_best.csv with side=7.343104
  N=184: saspav_best.csv with side=7.902634


In [7]:
# Create optimal valid baseline by picking best valid solution for each N
print("Creating optimal valid baseline by picking best valid solution for each N...\n")

sources = [
    '/home/code/preoptimized/best_snapshot.csv',
    '/home/code/preoptimized/better_ensemble.csv',
    '/home/code/preoptimized/bucket_of_chump.csv',
    '/home/code/preoptimized/chistyakov_best.csv',
    '/home/code/preoptimized/saspav_best.csv',
    '/home/code/preoptimized/submission_70_926.csv',
]

# Load all sources
all_trees = {}
all_sides = {}
for source in sources:
    name = source.split('/')[-1].replace('.csv', '')
    trees, sides = load_submission(source)
    all_trees[name] = trees
    all_sides[name] = sides

# For each N, pick the best valid solution
optimal_trees = {}
optimal_sides = {}
optimal_sources = {}

for n in range(1, 201):
    best_side = float('inf')
    best_source = None
    best_trees_n = None
    
    for name in all_trees:
        if not has_overlap(all_trees[name][n]):
            side = float(all_sides[name][n])
            if side < best_side:
                best_side = side
                best_source = name
                best_trees_n = all_trees[name][n]
    
    if best_source:
        optimal_trees[n] = best_trees_n
        optimal_sides[n] = Decimal(str(best_side))
        optimal_sources[n] = best_source
    else:
        print(f"WARNING: No valid solution found for N={n}")

optimal_score = calculate_score(optimal_sides)
print(f"Optimal valid baseline score: {optimal_score:.6f}")
print(f"Target: 68.896973")
print(f"Gap: {optimal_score - 68.896973:.6f} ({(optimal_score - 68.896973) / 68.896973 * 100:.2f}%)")

# Count sources used
from collections import Counter
source_counts = Counter(optimal_sources.values())
print(f"\nSources used:")
for source, count in source_counts.most_common():
    print(f"  {source}: {count} N values")

Creating optimal valid baseline by picking best valid solution for each N...



Optimal valid baseline score: 70.627582
Target: 68.896973
Gap: 1.730609 (2.51%)

Sources used:
  best_snapshot: 187 N values
  saspav_best: 10 N values
  bucket_of_chump: 2 N values
  better_ensemble: 1 N values


In [8]:
# Save optimal valid baseline
print("Saving optimal valid baseline...")

rows = []
for n in range(1, 201):
    trees = optimal_trees[n]
    for t_idx, tree in enumerate(trees):
        rows.append({
            'id': f'{n:03d}_{t_idx}',
            'x': f's{float(tree.center_x):.12f}',
            'y': f's{float(tree.center_y):.12f}',
            'deg': f's{float(tree.angle):.12f}'
        })

submission_df = pd.DataFrame(rows)
submission_df.to_csv('/home/code/preoptimized/optimal_valid_baseline.csv', index=False)
submission_df.to_csv('/home/submission/submission.csv', index=False)
print(f"Saved optimal valid baseline")

# Verify no overlaps
print("\nVerifying no overlaps...")
invalid_count = 0
for n in range(1, 201):
    if has_overlap(optimal_trees[n]):
        print(f"  N={n}: HAS OVERLAP")
        invalid_count += 1

if invalid_count == 0:
    print("  All N values are valid! No overlaps.")
else:
    print(f"  {invalid_count} N values have overlaps")

print(f"\nFinal score: {optimal_score:.6f}")
print(f"Target: 68.896973")
print(f"Gap: {optimal_score - 68.896973:.6f} ({(optimal_score - 68.896973) / 68.896973 * 100:.2f}%)")

Saving optimal valid baseline...
Saved optimal valid baseline

Verifying no overlaps...


  All N values are valid! No overlaps.

Final score: 70.627582
Target: 68.896973
Gap: 1.730609 (2.51%)
